## CASE STUDY - Deploying a recommender

We have seen the movie lens data on a toy dataset now lets try something a little bigger.  You have some
choices.

* [MovieLens Downloads](https://grouplens.org/datasets/movielens/latest/)

If your resources are limited (your working on a computer with limited amount of memory)

> continue to use the sample_movielens_ranting.csv

If you have a computer with at least 8GB of RAM

> download the ml-latest-small.zip

If you have the computational resources (access to Spark cluster or high-memory machine)

> download the ml-latest.zip

The two important pages for documentation are below.

* [Spark MLlib collaborative filtering docs](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html) 
* [Spark ALS docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS)


In [87]:
import os
import shutil
import pandas as pd
import numpy as np
import pyspark as ps
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.types import DoubleType

In [88]:
## ensure the spark context is available
spark = (ps.sql.SparkSession.builder
        .appName("sandbox")
        .getOrCreate()
        )

sc = spark.sparkContext
print(spark.version) 

2.4.5


### ensure the data are downloaded and specify the file paths here


In [89]:
data_dir = os.path.join(".", "ml-latest-small")
print(os.listdir(data_dir))

ratings_file = os.path.join(data_dir, "ratings.csv")
movies_file = os.path.join(data_dir, "movies.csv")

['links.csv', 'tags.csv', 'ratings.csv', 'README.txt', 'movies.csv']


In [90]:
## load the data
df_ratings = spark.read.format("csv").option("header", "true").option("inferSchema","true").load(ratings_file)
df_movies = spark.read.format("csv").option("header", "true").option("inferSchema","true").load(movies_file)

## QUESTION 1

Explore the movie lens data a little and summarize it

In [91]:
## YOUR CODE HERE (summarize the data)

df_ratings.show(n=4)
df_movies.show(n=4)

print(f'Ratings: {df_ratings.count()} records with {len(df_ratings.columns)} columns\n\n')
print(f'Movies: {df_movies.count()} records with {len(df_movies.columns)} columns\n\n')



+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
+------+-------+------+---------+
only showing top 4 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
+-------+--------------------+--------------------+
only showing top 4 rows

Ratings: 100836 records with 4 columns


Movies: 9742 records with 3 columns




## QUESTION 2

Find the ten most popular movies---that is the then movies with the highest average rating

>Hint: you may want to subset the movie matrix to only consider movies with a minimum number of ratings

In [93]:
## YOUR CODE HERE

from pyspark.sql import functions as F

df_ratings.groupBy("movieId") \
    .agg(F.avg("rating"), F.count("userId")) \
    .orderBy(["avg(rating)", "count(userId)"], ascending=[0,0]) \
    .show(10)

# SELECT movieId, AVG(rating), count(userId) GroupBy movieID  

+-------+-----------+-------------+
|movieId|avg(rating)|count(userId)|
+-------+-----------+-------------+
|  78836|        5.0|            2|
|   3473|        5.0|            2|
|   6818|        5.0|            2|
|     53|        5.0|            2|
|   6442|        5.0|            2|
|   1151|        5.0|            2|
|     99|        5.0|            2|
| 142444|        5.0|            1|
|  26350|        5.0|            1|
|    148|        5.0|            1|
+-------+-----------+-------------+
only showing top 10 rows



## QUESTION 3

Compare at least 5 different values for the ``regParam``

Use the `` ALS.trainImplicit()`` and compare it to the ``.fit()`` method.  See the [Spark ALS docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS)
for example usage. 

In [70]:
## YOUR CODE HERE

(training, test) = df_ratings.randomSplit([0.8, 0.2])

def train_eval_als(lambda_, implicit=False):
    als = ALS(rank=5, 
              maxIter=5, 
              regParam = lambda_, 
              userCol="userId", 
              itemCol="movieId", 
              ratingCol="rating", 
              coldStartStrategy="drop",
              implicitPrefs=implicit)
    model = als.fit(training)
    
    predictions = model.transform(test)
    
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    print(f'Lambda = {lambda_}: RMSE = {str(round(rmse,3))}')
    
    return model
    

for lambda_ in [0.001, 0.01, 0.1, 0.3, 0.5]:
    train_eval_als(lambda_)

Lambda = 0.001: RMSE = 1.124
Lambda = 0.01: RMSE = 1.015
Lambda = 0.1: RMSE = 0.9
Lambda = 0.3: RMSE = 0.926
Lambda = 0.5: RMSE = 1.004


## QUESTION 4

With your best `regParam` try using the `implicitPrefs` flag.

In [71]:
## YOUR CODE HERE

train_eval_als(0.1, True)

Lambda = 0.1: RMSE = 3.249


ALS_2aa0ebe85f8d

## QUESTION 5

Use model persistence to save your finalized model

In [72]:
## YOUR CODE HERE

model = train_eval_als(0.1)

save_dir = "saved-recommender"
if os.path.isdir(save_dir):
    print("overwriting saved model")
    shutil.rmtree(save_dir)

model.save(save_dir)


Lambda = 0.1: RMSE = 0.9


## QUESTION 6

Use ``spark-submit`` to load the model and demonstrate that you can load the model and interface with it.

In [83]:
%%writefile example-spark-submit.sh
/usr/local/bin/spark/spark-submit \
--master local[*] \
--executor-memory 1G \
--driver-memory 1G \
$@


Overwriting example-spark-submit.sh


In [86]:
!example-spark-submit.sh recommender-submit.py

/bin/sh: 1: example-spark-submit.sh: not found
